# `BlockScheme` to represent time integration methods

Time-stepping methods are represented by objects inheriting from a base class `BlockScheme` defined in the `blockops.schemes` submodule.
This class implements a `getBlockOperators(lamDt, phiName, chiName)` method that returns two `BlockOperator` objects that represent the $\phi$ and $\chi$ block operators for one block, such that:

$$
\phi {\bf u}_{n+1} = \chi {\bf u}_{n}.
$$

The `getBlockOperators` method uses two abstract method that should be overwritten in the children classes :

- `getBlockMatrices(lamDt)` : generate the block matrices for $\phi$ and $\chi$
- `getBlockCosts()` : estimate costs for $\phi$ and $\chi$

## Implementation and parameter documentation

All available time integration methods are registered in the `SCHEMES` dictionnary of the `blockops.schemes` submodule. This dictionnary contains all available classes that inherit from `BlockScheme` to implement one given type of time-stepping method.

In [1]:
from blockops.schemes import SCHEMES

SCHEMES

{'Collocation': blockops.schemes.coll.Collocation,
 'RungeKutta': blockops.schemes.rk.RungeKutta}

Here we can see that two time-stepping schemes are available, namely `Collocation` and `RungeKutta`.

If we look closely to the `RungeKutta` class, we can access all its parameters through its `PARAMS` attribute :

In [2]:
rk = SCHEMES['RungeKutta']

rk.PARAMS

{'nPoints': PositiveNumber(default=None),
 'ptsType': MultipleChoices(default=EQUID),
 'quadType': MultipleChoices(default=LOBATTO),
 'form': MultipleChoices(default=Z2N),
 'rkScheme': MultipleChoices(default=BE),
 'nStepsPerPoint': PositiveNumber(default=1)}

Each parameter is represented by a given `Parameter` class, which stores :

- the documentation for this parameter
- what kind of value is expected
- what are the current default for the class (`default=None` indicate that this parameter must be provided by the user)

For instance, if we look at the `ptsType` parameter :

In [3]:
ptsType = rk.PARAMS['ptsType']

print(ptsType.docs)

Either the type of points (EQUID, LEGENDRE), or a list of given time
points in [0, 1].


In [4]:
ptsType.__doc__

'Parameter that accepts different parameter values or parameter types'

In [5]:
ptsType.default

'EQUID'

In particular, for the `MultipleChoices` parameter class, the possible values and parameter types are stored in the `choices` and `pTypes` attributes :

In [6]:
ptsType.choices, ptsType.pTypes

(['EQUID', 'LEGENDRE', 'CHEBY-1', 'CHEBY-2', 'CHEBY-3', 'CHEBY-4'],
 [CustomPoints(default=None)])

In [7]:
ptsType.pTypes[0].__doc__

'Parameter that accepts an ordered list of float values in [0, 1]'

Back to the `RungeKutta` class, as for the `Collocation` class, it inherit from utility methods allowing to get the default and docs for each parameters :

In [8]:
rk.getParamsDefault()

{'nPoints': None,
 'ptsType': 'EQUID',
 'quadType': 'LOBATTO',
 'form': 'Z2N',
 'rkScheme': 'BE',
 'nStepsPerPoint': 1}

In [9]:
for param, doc in rk.getParamsDocs().items():
    print(f'--- {param} ---')
    print(doc)

--- nPoints ---
Number of time points in the block. Ignored if a custom list of points
is given for `ptsType`.
--- ptsType ---
Either the type of points (EQUID, LEGENDRE), or a list of given time
points in [0, 1].
--- quadType ---
Quadrature type used for the points in [0, 1]:
- LOBATTO -> 0 and 1 are included
- GAUSS -> neither 0 nor 1 are included
- RADAU-RIGHT -> only 1 is included
- RADAU-LEFT -> only 0 is included
--- form ---
Used formulation, either N2N (node-to-node) or Z2N (zero-to-node).
--- rkScheme ---
Name of the Runge-Kutta scheme (BE, FE, TRAP, RK4, ...).
--- nStepsPerPoint ---
Number of time-steps per block time point.
